<a href="https://colab.research.google.com/github/azizmehwish/45_assigment-_by_mehwish/blob/main/Langchain_Rag_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install pinecone-client
!pip install google-generativeai
!pip install openai
!pip install tqdm
!pip install langchain pinecone-client google-generativeai openai tqdm

In [ ]:

from google.colab import userdata
import os
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
PINECONE_ENVIRONMENT = 'us-east-1'


In [ ]:
!pip install python-dotenv


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec


pc = Pinecone(
    api_key=PINECONE_API_KEY
)

# Check if the index exists; if not, create it
index_name = "gemini-1-5-flash"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",  # Choose the metric: cosine, euclidean, or dotproduct
        spec=ServerlessSpec(
            cloud="aws",
            region=PINECONE_ENVIRONMENT  # Use your environment's region
        )
    )

# # Connect to the index
index = pc.Index(name=index_name)
print(f"Successfully connected to index: {index_name}")


Successfully connected to index: gemini-1-5-flash


In [ ]:
!pip install langchain-google-genai

In [ ]:
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY # Assuming PINECONE_API_KEY is already defined


embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",  # Specify the desired embedding model
    api_key=GOOGLE_API_KEY
)

In [ ]:
!pip install langchain-community


In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load documents
loader = TextLoader("/bin/X11/.document.txt")  # Replace with your file
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)


In [ ]:
from tqdm import tqdm

# Create embeddings and upload to Pinecone
for doc in tqdm(docs):
    vector = embeddings.embed_query(doc.page_content)

    # Modify the upsert to use a dictionary for metadata
    index.upsert([{
        "id": doc.metadata["source"],  # Use "id" instead of the first tuple element
        "values": vector,  # Use "values" for the vector
        "metadata": {
            "text": doc.page_content,  # Add the text as part of metadata
            "source": doc.metadata["source"]  # Include the source
        }
    }])

100%|██████████| 29/29 [00:07<00:00,  3.81it/s]


In [ ]:
from langchain.vectorstores import Pinecone

# Use from_existing_index to load from an existing index
retriever = Pinecone.from_existing_index(index_name=index_name, embedding=embeddings, text_key="text")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

gemini_model = ChatGoogleGenerativeAI(api_key=GOOGLE_API_KEY,model="gemini-1.5-flash", temperature=0.7)


In [ ]:
!pip install langchain-pinecone


In [ ]:

from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA

# Create a vector store using the Pinecone index
vectorstore = PineconeVectorStore(
    index_name=index_name,
    embedding=embeddings
)
# Create the retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})  # Retrieve top 4 most similar documents

# Create the QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=gemini_model,
    chain_type="stuff",
    retriever=retriever,  # Pass the retriever here
    return_source_documents=True  # Optional: to get the source documents used in the response
)



In [ ]:

print("Welcome To Mehwish's Coding World")
print('='*40)

query = "What is Agentic Artificial Intelligence?"
print('Human Message:',query)
response = qa_chain.invoke(query)

# Print the answer
print("Agent Message:", response['result'])

# Print the source documents
print("\n According To Given Information:")
for doc in response['source_documents']:
    print(f"- {doc.page_content[:200]}...")  # Print first 200 characters of each source document


Welcome To Mehwish's Coding World
Human Message: What is Agentic Artificial Intelligence?
Agent Message: Based on the provided text, I don't have enough information to define Agentic Artificial Intelligence.  The text only mentions it in passing as a field alongside Generative AI.


 According To Given Information:
- The progress in Generative AI and Agentic AI will lead to smarter, more adaptive systems that can autonomously generate so...
